### Data tunning

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
datos = pd.read_csv('/content/2.MOCK_DATA.csv')
datos = datos.sort_values(by=['hora']).reset_index(drop=True)

In [3]:
# Creo la columna 'consumo_del_dia' con datos coherentes
consumo = []
for i in range(1000):
  consumo.append(i/random.randrange(30,38) + i*0.01)
consumo.sort()
consumo = pd.DataFrame(consumo)
consumo = round(consumo,4)
datos = pd.concat([datos, consumo], axis=1)
datos.rename(columns = {0:'consumo_del_dia'}, inplace = True)

In [4]:
# Adecúo el campo 'hora' para que quede cada 30 segundos (como dice la letra)
datos['hora2'] = datos['hora'].str[:6]
secs = ['00','30']*500
datos['secs'] = secs
datos['hora_total'] = datos["hora2"] + datos["secs"]
datos = datos.loc[:, datos.columns != 'hora']
datos = datos.loc[:, datos.columns != 'hora2']
datos = datos.loc[:, datos.columns != 'secs']
datos.rename(columns = {'hora_total':'hora'}, inplace = True)
datos['fecha_hora'] = datos["fecha"] + ' ' + datos['hora']
datos = datos.loc[:, datos.columns != 'fecha']
datos = datos.loc[:, datos.columns != 'hora']

In [5]:
# Creo la columna 'bateria_resetante' con los datos de bateria1 y bateria2
bateria_1 = datos['bateria1'].head(500)
bateria_1 = pd.DataFrame(bateria_1).sort_values(by='bateria1', ascending=False).reset_index(drop=True)
bateria_1 = list(bateria_1['bateria1'])
bateria_2 = datos['bateria2'].head(500)
bateria_2 = pd.DataFrame(bateria_2).sort_values(by='bateria2', ascending=False).reset_index(drop=True)
bateria_2 = list(bateria_2['bateria2'])
bateria_restante = bateria_1 + bateria_2
bateria_restante = pd.DataFrame(bateria_restante).reset_index(drop=True)
datos['bateria'] = bateria_restante
datos = datos.loc[:, datos.columns != 'bateria1']
datos = datos.loc[:, datos.columns != 'bateria2']

In [6]:
# Creo la columna 'dist_supercharger' con los datos de dist_supercharger1 y dist_supercharger2
dist_1 = pd.DataFrame(datos['dist_supercharger1'])
dist_1 = dist_1.sort_values(by='dist_supercharger1', ascending=False)
dist_1 = dist_1.head(500).reset_index(drop=True)
dist_2 = pd.DataFrame(datos['dist_supercharger2'])
dist_2 = dist_2.sort_values(by='dist_supercharger2', ascending=False)
dist_2 = dist_2.head(500).reset_index(drop=True)

dist_1 = list(dist_1['dist_supercharger1'])
dist_2 = list(dist_2['dist_supercharger2'])
dist_supercharger = dist_1 + dist_2
datos['dist_supercharger'] = dist_supercharger
datos = datos.loc[:, datos.columns != 'dist_supercharger1']
datos = datos.loc[:, datos.columns != 'dist_supercharger2']

In [7]:
# Reordeno las columnas
datos = datos[['id_vehiculo', 'fecha_hora', 'ondas', 'temp_motor', 'vibracion', 'presion',
               'humedad','voltaje', 'velocidad','consumo_del_dia','bateria', 'dist_supercharger']]
datos          

### CSV

In [ ]:
datos.to_csv('/content/data_to_elastic.csv')

### JSON

In [ ]:
# https://appdividend.com/2022/03/15/pandas-to_json/#:~:text=To%20convert%20the%20object%20to,use%20the%20to_json()%20function.

datos.to_json('./data_to_elastic.json', orient='index')
datos2 = pd.read_json('/content/data_to_elastic.json')
datos2